### Import dependencies

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime as dt
from sklearn.metrics import confusion_matrix, classification_report

### Merge datasets

In [2]:
# Read in data
cpi = pd.read_csv("resources/cpi_final.csv")
gdp = pd.read_csv("resources/gdp_final.csv")
gdp_pct = pd.read_csv("resources/gdp_pct_chg_final.csv")
houst = pd.read_csv("resources/housing_starts_final.csv")
opg = pd.read_csv("resources/output_gap_final.csv")
rec_dt = pd.read_csv("resources/recession_dates_final.csv")
unrate = pd.read_csv("resources/unemployment_rate_final.csv")
fed_funds = pd.read_csv("resources/fed_funds_final.csv")
yield10_2 = pd.read_csv("resources/10YT_minus_2YT_final.csv")
fred = pd.read_csv("resources/FRED_data.csv")

In [3]:
# Combine all data sets into one data frame
dfs = [cpi, gdp, gdp_pct, houst, opg, rec_dt, unrate, fed_funds, yield10_2, fred]
df = reduce(lambda left,right: pd.merge(left,right,on=['quarter'],how='outer'), dfs)

In [4]:
# Drop date columns
df = df.drop(columns=['date_x','date_y'])

In [5]:
# Sort data frame by quarter
df = df.sort_values(by=['quarter'])

In [6]:
# Check dataset before removing nulls
# df.tail()

In [7]:
# Drop rows with missing values
df = df.dropna()

In [8]:
# Check dataset after removing nulls
df.tail()

,quarter,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,target,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,...,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures
284,2018Q1,249.250333,20163.159,5.0,1320.666667,0.202456,0.0,4.333333,1.448966,0.204683,...,-0.113861,6.9,1.8,2.214194,104.59493,3542.412,1.451,331800.0,4.597,13728.357
285,2018Q2,250.578667,20510.177,7.1,1259.666667,0.589182,0.0,3.833333,1.727176,0.192007,...,-0.251397,2.7,2.0,2.711887,103.33928,3561.592,1.461,315600.0,4.592,13939.828
286,2018Q3,251.828667,20749.752,4.8,1233.000000,0.821959,0.0,3.866667,1.923492,0.113663,...,-0.432836,3.3,2.0,2.640940,103.69309,3683.981,1.462,330900.0,4.587,14114.559
287,2018Q4,252.759000,20897.804,2.9,1185.000000,0.592021,0.0,3.566667,2.217097,0.152641,...,-0.078947,2.8,1.9,2.203131,105.15026,3725.234,1.462,322800.0,4.582,14211.920
288,2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,0.0,4.133333,2.401311,0.083088,...,-0.271429,4.5,1.6,1.644936,104.40334,3783.364,1.458,313000.0,4.577,14266.250


In [9]:
# Set index to quarter
df = df.set_index('quarter')

In [10]:
# Rename target column
df = df.rename(columns={'target':'recession_actual'})
df.head()

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,recession_actual,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,...,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures
quarter,,,,,,,,,,,,,,,,,,,,,
1976Q3,57.300000,1886.558,7.6,1557.000000,-2.199151,0.0,7.600000,5.283478,0.016956,0.100618,...,0.370833,3.2,6.0,5.518087,33.64333,328.307,1.717,44400.0,6.217,1158.806
1976Q4,58.133333,1934.273,10.5,1691.333333,-2.246705,0.0,7.333333,4.874239,-0.077456,0.211941,...,0.337386,2.6,6.0,5.069403,33.78753,337.650,1.699,45500.0,6.223,1192.408
1977Q1,59.200000,1988.648,11.7,1844.333333,-1.877175,0.0,8.233333,4.660667,-0.043817,0.148254,...,-0.095455,0.9,6.2,5.857741,33.65136,360.313,1.689,46300.0,6.227,1228.212
1977Q2,60.233333,2055.909,14.2,1918.666667,-0.776696,0.0,6.933333,5.157473,0.106595,0.332835,...,-0.052764,3.8,6.5,6.847698,32.80422,389.703,1.701,48900.0,6.232,1255.980
1977Q3,61.066667,2118.473,12.7,2009.000000,0.186001,0.0,6.800000,5.816413,0.127764,0.344309,...,-0.342175,5.7,6.6,6.682162,32.98791,414.134,1.713,48800.0,6.235,1286.905


### Shift data with sliding window technique

In [11]:
df['recession_1q_out'] = df['recession_actual'].shift(-1)
df['recession_2q_out'] = df['recession_actual'].shift(-2)
df['recession_4q_out'] = df['recession_actual'].shift(-4)

In [12]:
# Create three datasets -- 1 for each model (recession 1Qtr out, 2Qtrs out, 4Qtrs out)
df_q1 = df.drop(columns=['recession_2q_out','recession_4q_out','recession_actual'])
df_q2 = df.drop(columns=['recession_4q_out','recession_1q_out','recession_actual'])
df_q4 = df.drop(columns=['recession_1q_out','recession_2q_out','recession_actual'])

In [13]:
# Delete missing values
df_q1 = df_q1.dropna()
df_q2 = df_q2.dropna()
df_q4 = df_q4.dropna()

In [14]:
# Define y variables
y1 = df_q1['recession_1q_out']
y2 = df_q2['recession_2q_out']
y3 = df_q4['recession_4q_out']

In [15]:
# Drop target
df_q1 = df_q1.drop(columns=['recession_1q_out'])
df_q2 = df_q2.drop(columns=['recession_2q_out'])
df_q4 = df_q4.drop(columns=['recession_4q_out'])

In [16]:
# Define X
X_q1 = df_q1
X_q2 = df_q2
X_q4 = df_q4

### Split and scale data

In [17]:
# Split data into training and testing (stratify=None, shuffle=False)
X1_train, X1_test, y1_train, y1_test=train_test_split(X_q1, y1, train_size=0.8, random_state=42, shuffle=False)
X2_train, X2_test, y2_train, y2_test=train_test_split(X_q2, y2, train_size=0.8, random_state=42, shuffle=False)
X3_train, X3_test, y3_train, y3_test=train_test_split(X_q4, y3, train_size=0.8, random_state=42, shuffle=False)

In [18]:
# Create scaler object
X1_scaler = StandardScaler().fit(X1_train)
X2_scaler = StandardScaler().fit(X2_train)
X3_scaler = StandardScaler().fit(X3_train)

# X full scaler
X1_full_scaler = StandardScaler().fit(X_q1)
X2_full_scaler = StandardScaler().fit(X_q2)
X3_full_scaler = StandardScaler().fit(X_q4)

In [19]:
# Scale training data
X1_train_scaled = X1_scaler.transform(X1_train)
X2_train_scaled = X2_scaler.transform(X2_train)
X3_train_scaled = X3_scaler.transform(X3_train)

# Scale testing data
X1_test_scaled = X1_scaler.transform(X1_test)
X2_test_scaled = X2_scaler.transform(X2_test)
X3_test_scaled = X3_scaler.transform(X3_test)

# Scale full X data (no splits)
X1_full_scaled = X1_full_scaler.transform(X_q1)
X2_full_scaled = X2_full_scaler.transform(X_q2)
X3_full_scaled = X3_full_scaler.transform(X_q4)

### Reshape data to fit LSTM format

In [20]:
# Method to reshape data
def reshape_data(obj):
    reshaped_obj = np.reshape(obj, (obj.shape[0], obj.shape[1], 1))
    return reshaped_obj

In [21]:
# Reshape training data
reshaped_X1_train_scaled = reshape_data(X1_train_scaled)
reshaped_X2_train_scaled = reshape_data(X2_train_scaled)
reshaped_X3_train_scaled = reshape_data(X3_train_scaled)

In [22]:
# Reshape testing data
reshaped_X1_test_scaled = reshape_data(X1_test_scaled)
reshaped_X2_test_scaled = reshape_data(X2_test_scaled)
reshaped_X3_test_scaled = reshape_data(X3_test_scaled)

In [23]:
# Reshape X_full
reshaped_X1_full = reshape_data(X1_full_scaled)
reshaped_X2_full = reshape_data(X2_full_scaled)
reshaped_X3_full = reshape_data(X3_full_scaled)

# Build Model

In [24]:
# Initialize model
model = Sequential()

In [25]:
# Add layers
model.add(LSTM(128, input_shape=(reshaped_X1_train_scaled.shape[1],1), return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())  # Normalize activation outputs

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [26]:
# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

### Train and predict on X1-Y1 data (recession 1 quarter out)

In [27]:
# Fit the model to the training data
model.fit(reshaped_X1_train_scaled, y1_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 108 samples, validate on 28 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
108/108 - 16s - loss: 0.9347 - acc: 0.5093 - val_loss: 0.6840 - val_acc: 0.7500
Epoch 2/100
108/108 - 2s - loss: 0.6631 - acc: 0.6667 - val_loss: 0.6751 - val_acc: 0.7500
Epoch 3/100
108/108 - 2s - loss: 0.6772 - acc: 0.6667 - val_loss: 0.6656 - val_acc: 0.7500
Epoch 4/100
108/108 - 2s - loss: 0.4941 - acc: 0.7222 - val_loss: 0.6542 - val_acc: 0.7500
Epoch 5/100
108/108 - 2s - loss: 0.5376 - acc: 0.7407 - val_loss: 0.6432 - val_acc: 0.7500
Epoch 6/100
108/108 - 2s - loss: 0.5351 - acc: 0.7037 - val_loss: 0.6324 - val_acc: 0.7500
Epoch 7/100
108/108 - 2s - loss: 0.3947 - acc: 0.8148 - val_loss: 0.6239 - val_acc: 0.7500
Epoch 8/100
108/108 - 2s - loss: 0.4372 - acc: 0.8426 - val_loss: 0.6143 - val_acc: 0.7500
Epoch 9/100
108/108 - 2s - loss: 0.4535 - acc: 0.7778 - val_loss: 0.6068 - val_acc: 0.7500
Epoch 10/100
108/108 - 2s - loss: 0.4990 

108/108 - 2s - loss: 0.0832 - acc: 0.9815 - val_loss: 1.1384 - val_acc: 0.4643
Epoch 87/100
108/108 - 2s - loss: 0.1321 - acc: 0.9352 - val_loss: 1.1238 - val_acc: 0.5357
Epoch 88/100
108/108 - 2s - loss: 0.1027 - acc: 0.9630 - val_loss: 1.0854 - val_acc: 0.5000
Epoch 89/100
108/108 - 2s - loss: 0.0864 - acc: 0.9630 - val_loss: 0.9211 - val_acc: 0.6071
Epoch 90/100
108/108 - 2s - loss: 0.1126 - acc: 0.9722 - val_loss: 0.9103 - val_acc: 0.6071
Epoch 91/100
108/108 - 2s - loss: 0.0954 - acc: 0.9630 - val_loss: 0.9496 - val_acc: 0.5714
Epoch 92/100
108/108 - 2s - loss: 0.0934 - acc: 0.9630 - val_loss: 1.1126 - val_acc: 0.5357
Epoch 93/100
108/108 - 2s - loss: 0.0809 - acc: 0.9722 - val_loss: 1.2468 - val_acc: 0.5000
Epoch 94/100
108/108 - 2s - loss: 0.0732 - acc: 0.9815 - val_loss: 1.3348 - val_acc: 0.4643
Epoch 95/100
108/108 - 2s - loss: 0.0684 - acc: 1.0000 - val_loss: 1.3977 - val_acc: 0.4286
Epoch 96/100
108/108 - 2s - loss: 0.1160 - acc: 0.9722 - val_loss: 1.3318 - val_acc: 0.4643
E

In [28]:
# Evaluate model using test data
model_loss1, model_accuracy1 = model.evaluate(reshaped_X1_test_scaled, y1_test, verbose=2)

34/34 - 0s - loss: 0.1257 - acc: 0.9412


In [29]:
# Make predictions using test data
predictions1 = model.predict_classes(reshaped_X1_test_scaled)

In [31]:
# Compare results
one_qtr_out = pd.DataFrame({"Predicted":predictions1, "Actual":y1_test})
# one_qtr_out

In [32]:
# Save model
name1 = f"unshuffled-1q-out-{dt.datetime.now()}"
model.save(f"models/{name1}.h5")

#### Predict on full X1

In [33]:
pred_X1_full = model.predict_classes(reshaped_X1_full)

# Preview results
X1_full_results = pd.DataFrame({"Predicted":pred_X1_full, "Actual":y1})
# X1_full_results.loc[X1_full_results["Actual"]==1]

# Export results for graphing
X1_full_results.to_csv(f"resources/predictions/X1_full_unshuffled_{dt.datetime.now()}.csv")

#### Confusion Matrix on X1-Y1 data (recession 1 quarter out)

In [34]:
# Create confusion matrix on X1 model
con_mat = confusion_matrix(y1_test, predictions1)
print(con_mat)

[[32  2]
 [ 0  0]]


In [35]:
# Score model
print(classification_report(y1_test, predictions1))

              precision    recall  f1-score   support

         0.0       1.00      0.94      0.97        34
         1.0       0.00      0.00      0.00         0

    accuracy                           0.94        34
   macro avg       0.50      0.47      0.48        34
weighted avg       1.00      0.94      0.97        34



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Train and predict on X2-Y2 data (recession 2 quarters out)

In [36]:
# Fit the model to the training data
model.fit(reshaped_X2_train_scaled, y2_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 108 samples, validate on 27 samples
Epoch 1/100
108/108 - 2s - loss: 0.3638 - acc: 0.9074 - val_loss: 1.2643 - val_acc: 0.4444
Epoch 2/100
108/108 - 2s - loss: 0.2451 - acc: 0.9259 - val_loss: 1.4562 - val_acc: 0.4444
Epoch 3/100
108/108 - 2s - loss: 0.1438 - acc: 0.9352 - val_loss: 1.8528 - val_acc: 0.3704
Epoch 4/100
108/108 - 2s - loss: 0.2058 - acc: 0.9259 - val_loss: 2.1940 - val_acc: 0.2222
Epoch 5/100
108/108 - 2s - loss: 0.1258 - acc: 0.9444 - val_loss: 2.0669 - val_acc: 0.2963
Epoch 6/100
108/108 - 2s - loss: 0.1435 - acc: 0.9167 - val_loss: 1.7376 - val_acc: 0.3704
Epoch 7/100
108/108 - 2s - loss: 0.1205 - acc: 0.9444 - val_loss: 1.5341 - val_acc: 0.4815
Epoch 8/100
108/108 - 2s - loss: 0.0825 - acc: 0.9815 - val_loss: 1.4467 - val_acc: 0.4815
Epoch 9/100
108/108 - 2s - loss: 0.1156 - acc: 0.9537 - val_loss: 1.3789 - val_acc: 0.5556
Epoch 10/100
108/108 - 2s - loss: 0.0832 - acc: 0.9722 - val_loss: 1.3687 - val_acc: 0.5185
Epoch 11/100
108/108 - 2s - loss: 0.0823 - a

Epoch 90/100
108/108 - 2s - loss: 0.0060 - acc: 1.0000 - val_loss: 1.5629 - val_acc: 0.6296
Epoch 91/100
108/108 - 2s - loss: 0.0062 - acc: 1.0000 - val_loss: 1.5409 - val_acc: 0.6296
Epoch 92/100
108/108 - 2s - loss: 0.0075 - acc: 1.0000 - val_loss: 1.5171 - val_acc: 0.6296
Epoch 93/100
108/108 - 2s - loss: 0.0116 - acc: 1.0000 - val_loss: 1.5141 - val_acc: 0.6296
Epoch 94/100
108/108 - 2s - loss: 0.0071 - acc: 1.0000 - val_loss: 1.5103 - val_acc: 0.6296
Epoch 95/100
108/108 - 2s - loss: 0.0141 - acc: 1.0000 - val_loss: 1.5316 - val_acc: 0.6667
Epoch 96/100
108/108 - 2s - loss: 0.0132 - acc: 0.9907 - val_loss: 1.4724 - val_acc: 0.6667
Epoch 97/100
108/108 - 2s - loss: 0.0160 - acc: 1.0000 - val_loss: 1.4180 - val_acc: 0.7037
Epoch 98/100
108/108 - 2s - loss: 0.0634 - acc: 0.9815 - val_loss: 1.5531 - val_acc: 0.6667
Epoch 99/100
108/108 - 2s - loss: 0.0239 - acc: 0.9907 - val_loss: 1.4497 - val_acc: 0.7037
Epoch 100/100
108/108 - 2s - loss: 0.0517 - acc: 0.9722 - val_loss: 1.2008 - val

In [37]:
# Evaluate model using test data
model_loss2, model_accuracy2 = model.evaluate(reshaped_X2_test_scaled, y2_test, verbose=2)

34/34 - 0s - loss: 0.0561 - acc: 1.0000


In [38]:
# Make predictions using test data
predictions2 = model.predict_classes(reshaped_X2_test_scaled)

In [39]:
# Compare results
two_qtrs_out = pd.DataFrame({"Predicted":predictions2, "Actual":y2_test})
# two_qtrs_out

In [40]:
# Save model
name2 = f"unshuffled-2q-out-{dt.datetime.now()}"
model.save(f"models/{name2}.h5")

#### Predict on full X2

In [41]:
pred_X2_full = model.predict_classes(reshaped_X2_full)

# Preview results
X2_full_results = pd.DataFrame({"Predicted":pred_X2_full, "Actual":y2})
# X2_full_results.loc[X2_full_results["Actual"]==1]

# Export results for graphing
X2_full_results.to_csv(f"resources/predictions/X2_full_unshuffled_{dt.datetime.now()}.csv")

#### Confusion Matrix on X2-Y2 data (recession 2 quarters out)

In [42]:
# Create confusion matrix on X2 model
con_mat = confusion_matrix(y2_test, predictions2)
print(con_mat)

[[34]]


In [43]:
# Score model
print(classification_report(y2_test, predictions2))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        34

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



### Train and predict on X3-Y3 data (recession 4 quarters out)

In [44]:
# Fit the model to the training data
model.fit(reshaped_X3_train_scaled, y3_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 106 samples, validate on 27 samples
Epoch 1/100
106/106 - 2s - loss: 1.0295 - acc: 0.8491 - val_loss: 0.8128 - val_acc: 0.5556
Epoch 2/100
106/106 - 2s - loss: 0.4787 - acc: 0.8962 - val_loss: 0.9086 - val_acc: 0.4074
Epoch 3/100
106/106 - 2s - loss: 0.4819 - acc: 0.8585 - val_loss: 0.9920 - val_acc: 0.4444
Epoch 4/100
106/106 - 2s - loss: 0.1666 - acc: 0.9245 - val_loss: 1.1620 - val_acc: 0.4815
Epoch 5/100
106/106 - 2s - loss: 0.2563 - acc: 0.9151 - val_loss: 1.3065 - val_acc: 0.4074
Epoch 6/100
106/106 - 2s - loss: 0.1476 - acc: 0.9528 - val_loss: 1.3921 - val_acc: 0.4074
Epoch 7/100
106/106 - 2s - loss: 0.1695 - acc: 0.9434 - val_loss: 1.2403 - val_acc: 0.4815
Epoch 8/100
106/106 - 2s - loss: 0.2333 - acc: 0.8962 - val_loss: 1.2374 - val_acc: 0.5556
Epoch 9/100
106/106 - 2s - loss: 0.1311 - acc: 0.9623 - val_loss: 1.2849 - val_acc: 0.4815
Epoch 10/100
106/106 - 2s - loss: 0.1346 - acc: 0.9434 - val_loss: 1.3852 - val_acc: 0.4074
Epoch 11/100
106/106 - 2s - loss: 0.1619 - a

Epoch 90/100
106/106 - 2s - loss: 0.0204 - acc: 0.9906 - val_loss: 2.1752 - val_acc: 0.5556
Epoch 91/100
106/106 - 2s - loss: 0.0126 - acc: 1.0000 - val_loss: 2.4613 - val_acc: 0.5185
Epoch 92/100
106/106 - 2s - loss: 0.0092 - acc: 1.0000 - val_loss: 2.7232 - val_acc: 0.4444
Epoch 93/100
106/106 - 2s - loss: 0.0102 - acc: 1.0000 - val_loss: 2.8377 - val_acc: 0.4444
Epoch 94/100
106/106 - 2s - loss: 0.0145 - acc: 1.0000 - val_loss: 2.6451 - val_acc: 0.4815
Epoch 95/100
106/106 - 2s - loss: 0.0105 - acc: 1.0000 - val_loss: 2.2842 - val_acc: 0.4815
Epoch 96/100
106/106 - 2s - loss: 0.0034 - acc: 1.0000 - val_loss: 2.0421 - val_acc: 0.4815
Epoch 97/100
106/106 - 2s - loss: 0.0041 - acc: 1.0000 - val_loss: 1.8979 - val_acc: 0.5556
Epoch 98/100
106/106 - 2s - loss: 0.0101 - acc: 1.0000 - val_loss: 1.8491 - val_acc: 0.5556
Epoch 99/100
106/106 - 2s - loss: 0.0042 - acc: 1.0000 - val_loss: 1.8429 - val_acc: 0.5556
Epoch 100/100
106/106 - 2s - loss: 0.0094 - acc: 1.0000 - val_loss: 1.9031 - val

In [45]:
# Evaluate model using test data
model_loss3, model_accuracy3 = model.evaluate(reshaped_X3_test_scaled, y3_test, verbose=2)

34/34 - 0s - loss: 4.7858 - acc: 0.3529


In [46]:
# Make predictions using test data
predictions3 = model.predict_classes(reshaped_X3_test_scaled)

In [47]:
# Compare results
four_qtrs_out = pd.DataFrame({"Predicted":predictions3, "Actual":y3_test})
# four_qtrs_out

In [48]:
# Save model
name3 = f"unshuffled-4q-out-{dt.datetime.now()}"
model.save(f"models/{name3}.h5")

#### Predict on full X3

In [49]:
pred_X3_full = model.predict_classes(reshaped_X3_full)

# Preview results
X3_full_results = pd.DataFrame({"Predicted":pred_X3_full, "Actual":y3})
# X3_full_results.loc[X3_full_results["Actual"]==1]

# Export results for graphing
X3_full_results.to_csv(f"resources/predictions/X3_full_unshuffled_{dt.datetime.now()}.csv")

#### Confusion Matrix on X3-Y3 data (recession 4 quarters out)

In [50]:
# Create confusion matrix on X3 model
con_mat = confusion_matrix(y3_test, predictions3)
print(con_mat)

[[12 22]
 [ 0  0]]


In [51]:
# Score model
print(classification_report(y3_test, predictions3))

              precision    recall  f1-score   support

         0.0       1.00      0.35      0.52        34
         1.0       0.00      0.00      0.00         0

    accuracy                           0.35        34
   macro avg       0.50      0.18      0.26        34
weighted avg       1.00      0.35      0.52        34



# Predict on 2019 Data

In [ ]:
# 2019Q1: 1 Quarter Out

# 2019Q1: 2 Quarters Out

# 2019Q1: 4 Quarters Out

In [ ]:
# 2019Q2: 1 Quarter Out

# 2019Q2: 2 Quarters Out

# 2019Q2: 4 Quarters Out